<a href="https://colab.research.google.com/github/SolomonAYu/gallery/blob/main/%E3%80%8CAlpaca101%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Lighter Model - LLama

[Alpaca-LoRA](https://github.com/tloen/alpaca-lora/) for demonstration purposes.

# 新段落

In [ ]:
!pip install bitsandbytes
!pip install -q datasets loralib sentencepiece
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip install -q git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 19.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.5 MB/s eta 0:00:00
     

In [ ]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig

tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")
model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, "tloen/alpaca-lora-7b")

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [ ]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

In [ ]:
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

def evaluate(instruction, input=None):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        print("Response:", output.split("### Response:")[1].strip())
        code1 = output.split("### Response:")[1].strip()
    return code1

In [ ]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context.
              Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

In [ ]:
code = evaluate(input("Instruction: "))
code1 = ""

In [ ]:
generated_code_error=[]

In [ ]:
def evaluate_auto(instruction, input=None):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256,

    )
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        if len(output.split("### Response:")) == 1:
          generated_code_error.append(output.split("### Response:"))
        else:
          #print("Response:", output.split("### Response:")[1].strip())
          code1 = output.split("### Response:")[1].strip()
          return code1

In [ ]:
import json
with open('w3resource_pairs.json' ,'r') as f:
  pairs = json.load(f)

In [ ]:
id=0
for pair in pairs:
  pair['uid']=id
  id=id+1

In [ ]:
with open('original_data.json', 'w') as f:
    json.dump(pairs, f)

In [ ]:
generated_code = []

data_set = pairs[:400]
for pair in data_set:
  description = ' '.join(pair['description'])
  code = evaluate_auto(description)
  generated_code.append({'code': code, 'uid': pair['uid']})

In [ ]:
correct_output=[]
original_output=[]
for pair in data_set:
    correct_output.append(pair['output'])
    original_output.append(pair['code'])

In [ ]:
print(data_set)

[{'section': '72', 'part': 1, 'description': ['Write a Python program to flatten a given nested list structure.'], 'code': ['def flatten_list(nums):', '  result = []', '  for i in nums:', '    result.extend(i) if isinstance(i, list) else result.append(i)', '  return result', 'n_list = [0, 10, [20, 30], 40, 50, [60, 70, 80], [90, 100, 110, 120]]', 'print("Original list:")', 'print(n_list)', 'print("\\nFlatten list:")', 'print(flatten_list(n_list))'], 'lab': 9, 'topic': 'Python Data Types - List', 'output': ['Original list:', '[0, 10, [20, 30], 40, 50, [60, 70, 80], [90, 100, 110, 120]]', 'Flatten list:', '[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120]'], 'uni': 'w3resource', 'source': 'https://www.w3resource.com/python-exercises/list/python-data-type-list-exercise-72.php', 'uid': 800}, {'section': '73', 'part': 0, 'description': ['Write a Python program to remove consecutive (following each other continuously) duplicates (elements) from a given list.'], 'code': ['from itertools 

In [ ]:
len(data_set)

400

In [ ]:
print(generated_code)

[{'code': 'def flatten_list(nested_list):\n    flat_list = []\n    for item in nested_list:\n        if isinstance(item, list):\n            flat_list.extend(flatten_list(item))\n        else:\n            flat_list.append(item)\n    return flat_list', 'uid': 800}, {'code': 'def remove_duplicates(lst):\n    seen = set()\n    for i in lst:\n        if i not in seen:\n            seen.add(i)\n            lst.remove(i)\n    return lst', 'uid': 801}, {'code': 'def pack_duplicates(lst): \n    result = [] \n    for i in range(len(lst)-1): \n        if lst[i] == lst[i+1]: \n            result.append([lst[i]]) \n        else: \n            result.append([lst[i]]) \n            result.append([lst[i+1]]) \n    return result', 'uid': 802}, {'code': '"""\nRun-length encoding (RLE) is a form of lossless data compression in which runs of data (sequences in which the same data value occurs in many consecutive data elements) are stored as a single data value and count, rather than as the original run.

In [ ]:
len(generated_code)

400

In [ ]:
with open('data.json', 'w') as f:
    json.dump(generated_code, f)

In [ ]:
uno = 0
PE=[]
SR=[]
SO=[]
DO=[]
SC=[]
DC=[]
EOP=[]

In [ ]:
import signal

class TimeoutError(Exception):
    pass
def timeout_handler(signum, frame):
    raise TimeoutError("Code execution time exceeded.")

In [ ]:
def detect_deadloop(code_str, max_execution_time=3):
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(max_execution_time)
    flag = False
    try:
        exec(code_str)
        flag = False
    except TimeoutError:
        flag = True
    finally:
        signal.alarm(0)
        return flag

In [ ]:
import re
import traceback
for code in generated_code:
    if code['code'] is None:
        EOP.append(code)
    pattern = r"\binput\b"
    if re.search(pattern, code['code']):
        continue
    if detect_deadloop(code['code']):
      PE.append([code['code'], code['uid'], 'Deadloop', 'Infinite loop'])
      continue
    try:
        co = exec(code['code'])
    except Exception as e:
        PE.append([code['code'], code['uid'], str(type(e)), str(traceback.format_exc())])
    else:
        SR.append(code)


In [ ]:
print(len(EOP))

0


In [ ]:
print(SR)

[{'code': "def is_list(x):\n    return isinstance(x, list)\n\ndef is_tuple(x):\n    return isinstance(x, tuple)\n\ndef is_set(x):\n    return isinstance(x, set)\n\nif __name__ == '__main__':\n    x = [1, 2, 3]\n    print(is_list(x))\n    print(is_tuple(x))\n    print(is_set(x))", 'uid': 217}, {'code': '"""\nWrite a Python program that prints long text, converts it to a list, and prints all the words and the frequency of each word.\n"""\n\ndef main():\n    text = "The quick brown fox jumps over the lazy dog"\n    words = text.split()\n    word_list = list(words)\n    word_count = dict()\n    for word in word_list:\n        if word in word_count:\n            word_count[word] += 1\n        else:\n            word_count[word] = 1\n\n    for word, count in word_count.items():\n        print(word, count)\n\nif __name__ == \'__main__\':\n    main()', 'uid': 232}]


In [ ]:
len(SR)

293

In [ ]:
import subprocess

def execute_code(code_string):
    result = subprocess.run(['python', '-c', code_string], capture_output=True, text=True)
    output_result = result.stdout.strip()

    return output_result


In [ ]:
print(PE)

[['print(os.path.splitext(filename)[1])', 6, "<class 'NameError'>", 'Traceback (most recent call last):\n  File "<ipython-input-44-b307aedef65c>", line 13, in <cell line: 3>\n    co = exec(code[\'code\'])\n  File "<string>", line 1, in <module>\nNameError: name \'os\' is not defined\n'], ['for i in range(len(color_list)-1):\n    print(color_list[i], end=" ")\nprint(color_list[len(color_list)-1])', 7, "<class 'NameError'>", 'Traceback (most recent call last):\n  File "<ipython-input-44-b307aedef65c>", line 13, in <cell line: 3>\n    co = exec(code[\'code\'])\n  File "<string>", line 1, in <module>\nNameError: name \'color_list\' is not defined\n'], ['"""\nWrite a Python program to print the following \'here document\'.\nSample string: a string that you "don\'t" have to escape This is a....... multi-line heredoc string --------> example\n"""\n\ndef print_heredoc(heredoc):\n    for line in heredoc.split("\\n"):\n        print(line)\n\nprint_heredoc("""a string that you "don\'t" have to es

In [ ]:
len(PE)

75

In [ ]:
#SO&DO classifier
def output_type_classifier(o_output_list, g_output_list):
  for g in g_output_list:
    output=execute_code(g['code'])
    dicts = [d for d in o_output_list if d.get('uid') == g['uid']]
    if output == ''.join(dicts[0]['output']):
      SO.append(g)
    else:
      DO.append(g)

In [ ]:
#SC&DC classifier
def so_classifier(o_code_list, g_code_list):
  for g in g_code_list:
    new_string = (g['code']).replace(" ", "")
    my_list = (new_string).split("\n")
    dicts = [d for d in o_code_list if d.get('uid') == g['uid']]
    if (''.join(my_list)) == (''.join(dicts[0]['code'])):
      SC.append(g)
    else:
      DC.append(g)

In [ ]:
obj = {}
for i in PE:
    if i[2] not in obj:
        obj[i[2]] = []
    obj[i[2]].append({'code': i[0], 'uid': i[1], 'message': i[3]})

with open('error_out.json', 'w') as f:
    f.write(json.dumps(obj, indent=2))

In [ ]:
print(obj)

{"<class 'SyntaxError'>": [{'code': '"""\nWrite a Python program to create a two-dimensional list from a given list of lists.\n"""\n\nnums = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\n\ndef transpose(lst):\n    return list(zip(*lst))\n\ntransposed_nums = transpose(nums)\n\nprint(transposed_nums)\n\n# Output:\n[((1, 0), (2, 1), (3, 2), (4, 3), (5, 4), (6, 5), (7, 6), (8, 7), (9, 8), (10, 9)), (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), (1, 2, 3, 4, 5, 6, 7, 8, 9, 10), (2, 3, 4, 5, 6, 7, 8, 9,', 'uid': 1000, 'message': 'Traceback (most recent call last):\n  File "<ipython-input-14-b307aedef65c>", line 13, in <cell line: 3>\n    co = exec(code[\'code\'])\n  File "<string>", line 15\n    [((1, 0), (2, 1), (3, 2), (4, 3), (5, 4), (6, 5), (7, 6), (8, 7), (9, 8), (10, 9)), (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), (1, 2, 3, 4, 5, 6, 7, 8, 9, 10), (2, 3, 4, 5, 6, 7, 8, 9,\n                                                                                                                                                         ^

In [ ]:
output_type_classifier(data_set, SR)

In [ ]:
so_classifier(data_set, SO)

In [ ]:
len(DO)

289

In [ ]:
len(SO)

4

In [ ]:
with open('SR.json', 'w') as f:
    json.dump(SR, f)

In [ ]:
with open('SO.json', 'w') as f:
    json.dump(SO, f)

In [ ]:
with open('DO.json', 'w') as f:
    json.dump(DO, f)

In [ ]:
with open('SC.json', 'w') as f:
    json.dump(SC, f)

In [ ]:
with open('DC.json', 'w') as f:
    json.dump(DC, f)

In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from google.auth import default
creds, _ = default()

In [ ]:
client = gspread.authorize(creds)


spreadsheet_id = '1BKNwOSVmxVOj3Rhf_XbyZkKPcUlq6_cI19PFuAa3Zd4'
sheet_name = 'error'

# Write data to Google Sheets
worksheet = client.open_by_key(spreadsheet_id).worksheet(sheet_name)

# Clear the existing data in the sheet
worksheet.clear()

{'spreadsheetId': '1BKNwOSVmxVOj3Rhf_XbyZkKPcUlq6_cI19PFuAa3Zd4',
 'clearedRange': 'error!A1:Z1000'}

In [ ]:
print(pairs[1088]['output'])

['Original strings:', "['S001', 'S002', 'S003', 'S004']", "['Adina Park', 'Leyton Marsh', 'Duncan Boyle', 'Saim Richards']", '[85, 98, 89, 92]', 'Nested dictionary:', "[{'S001': {'Adina Park': 85}}, {'S002': {'Leyton Marsh': 98}}, {'S003': {'Duncan Boyle': 89}}, {'S004': {'Saim Richards': 92}}]"]


In [ ]:
print(DO[71])

{'code': 'def convert_lists_to_nested_dict(lists):\n    nested_dict = {}\n    for list in lists:\n        nested_dict[list[0]] = list[1:]\n    return nested_dict', 'uid': 1088}


In [ ]:
print(len(PE))
print(len(SR))
print(len(SO))
print(len(DO))
print(len(DC))
print(len(SC))

53
195
0
195
0
0


In [ ]:
import json

data1 = json.loads(open('error_out.json').read())

In [ ]:
# Prepare data to be written
rows_to_write = []

# Get the longest list of errors
max_len = max(len(error_list) for error_list in data1.values())

for i in range(max_len + 1):
    row = []
    for error_type, error_list in data1.items():
        if i == 0:
            # Write error type in the first row
            row.append(error_type)
        else:
            # Write error details in the following rows
            if i - 1 < len(error_list):
                error = error_list[i - 1]
                cell_content = f"code: {error['code']}, index: {error['index']}, message: {error['message']}"
                row.append(cell_content)
            else:
                row.append('')
    rows_to_write.append(row)

# Update the sheet with new data
worksheet.update('A1', rows_to_write)

{'spreadsheetId': '1P9s5S8z8rnHQX6GjW3ojjt3WvDGjs0qFwiooM_RZzCg',
 'updatedRange': 'error!A1:F28',
 'updatedRows': 28,
 'updatedColumns': 6,
 'updatedCells': 168}

In [ ]:
def chars(s):
    table = {
        '\\': '\\\\',
        '\n': '\\n',
        '\t': '\\t',
        '\r': '\\r',
        '\f': '\\f',
        '\v': '\\v',
        '\a': '\\a',
        '\b': '\\b',
        '\'': '\\\'',
        '\"': '\\\"',
    }
    return ''.join(table.get(c, c) for c in s)
a = chars(code)
print(a)

In [ ]:
import portpicker
import threading
import socket
import IPython

from google.colab import output
from six.moves import socketserver
from six.moves import SimpleHTTPServer


class V6Server(socketserver.TCPServer):
  address_family = socket.AF_INET6

class Handler(SimpleHTTPServer.SimpleHTTPRequestHandler):
  def do_GET(self):

    self.send_response(200)
    # If the response should not be cached in the notebook for
    # offline access:
    # self.send_header('x-colab-notebook-cache-control', 'no-cache')
    self.end_headers()

    html_a = '''
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>Simple BlockMirror Example</title>

    <script src="https://code.jquery.com/jquery-3.4.1.slim.min.js"
            integrity="sha256-pasqAKBDmFT4eHoN2ndd6lN370kFiGUFyTiUHWhU7k8="
            crossorigin="anonymous"></script>

    <!-- Blockly -->
    <script src="https://blockpy-edu.github.io/BlockMirror/lib/blockly/blockly_compressed.js" type="text/javascript"></script>
    <script src="https://blockpy-edu.github.io/BlockMirror/lib/blockly/blocks_compressed.js" type="text/javascript"></script>
    <script src="https://blockpy-edu.github.io/BlockMirror/lib/blockly/msg/js/en.js" type="text/javascript"></script>
    <script src="https://blockpy-edu.github.io/BlockMirror/lib/blockly/python_compressed.js" type="text/javascript"></script>

    <!-- CodeMirror -->
    <link rel="stylesheet" href="https://blockpy-edu.github.io/BlockMirror/lib/codemirror/codemirror.css">
    <link rel="stylesheet" href="https://blockpy-edu.github.io/BlockMirror/lib/codemirror/fullscreen.css">
    <link rel="stylesheet" href="https://blockpy-edu.github.io/BlockMirror/lib/codemirror/show-hint.css">
    <script src="https://blockpy-edu.github.io/BlockMirror/lib/codemirror/codemirror.js" type="text/javascript"></script>
    <script src="https://blockpy-edu.github.io/BlockMirror/lib/codemirror/show-hint.js" type="text/javascript"></script>
    <script src="https://blockpy-edu.github.io/BlockMirror/lib/codemirror/python-hint.js" type="text/javascript"></script>
    <script src="https://blockpy-edu.github.io/BlockMirror/lib/codemirror/fullscreen.js" type="text/javascript"></script>
    <script src="https://blockpy-edu.github.io/BlockMirror/lib/codemirror/python.js" type="text/javascript"></script>

    <!-- Skulpt -->
    <script src="https://blockpy-edu.github.io/BlockMirror/dist/skulpt_parser.js" type="text/javascript"></script>

    <!-- BlockMirror -->
    <link rel="stylesheet" href="https://blockpy-edu.github.io/BlockMirror/dist/block_mirror.css">
    <script src="https://blockpy-edu.github.io/BlockMirror/dist/block_mirror.js" type="text/javascript"></script>

</head>
<body>
<div id="image-spot"></div>
<div style="width:900px; margin:0 auto;"
     id='example-frame'>

    <div class="btn-group" data-toggle="buttons" data-bind="visible: !assignment.upload()">
        <label onclick="editor.setMode('block');">
            <input type="radio" name="blockpy-mode-set" autocomplete="off"> Blocks
        </label>
        <label onclick="editor.setMode('split');">
            <input type="radio" name="blockpy-mode-set" autocomplete="off" checked> Split
        </label>
        <label onclick="editor.setMode('text');">
            <input type="radio" name="blockpy-mode-set" autocomplete="off"> Text
        </label>
    </div>

    <div id="blockmirror-editor"></div>
</div>


<script type="text/javascript">
    var editor = new BlockMirror({{
        'container': document.getElementById('blockmirror-editor'),
        blocklyMediaPath: 'https://blockpy-edu.github.io/BlockMirror/lib/blockly/media/',

    }});

    editor.setCode('{}');

    editor.addChangeListener(function (event) {{
        console.log('Change! Better save:', event);
        var so = editor.getCode();

    }});


</script>

</body>
</html>
'''.format(a)


    html_a = str.encode(html_a)


    self.wfile.write(html_a)
port = portpicker.pick_unused_port()

def server_entry():
    httpd = V6Server(('::', port), Handler)
    # Handle a single request then exit the thread.
    httpd.serve_forever()

thread = threading.Thread(target=server_entry)
thread.start()

# Display some HTML referencing the resource.
display(IPython.display.HTML('<script src="https://localhost:{port}/"></script>'.format(port=port)))

In [ ]:
import IPython
import uuid
from google.colab import output

class InvokeButton(object):
  def __init__(self, title, callback, arg=None):
    self._title = title
    self._callback = callback
    self._arg = arg

  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)

    template = """<button id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', ['{arg}'], {{}})
            e.preventDefault();
          }};


        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, arg=self._arg)
    return html


In [ ]:
def Save(arg1):
   arg1 = a

In [ ]:
from google.colab import output
output.register_callback('evaluate', evaluate)
output.register_callback('Save', Save)

In [ ]:
from google.colab import output
output.serve_kernel_port_as_iframe(port)

<IPython.core.display.Javascript object>

In [ ]:
code2 = a + "is this code a valiad python code?" #Test

In [ ]:
InvokeButton('Send', evaluate, arg=code2)

Response: Yes, this is a valid Python code.


In [ ]:
from google.colab import output
output.serve_kernel_port_as_window(port)

<IPython.core.display.Javascript object>